In [2]:
!pip install aiohttp~=3.6.2

     |████████████████████████████████| 1.2 MB 3.1 MB/s eta 0:00:01
     |████████████████████████████████| 256 kB 4.4 MB/s eta 0:00:01
     |████████████████████████████████| 149 kB 1.2 MB/s eta 0:00:01


In [3]:
!pip install PyPubSub

     |████████████████████████████████| 61 kB 1.1 MB/s eta 0:00:011


In [4]:
!pip install asyncio

     |████████████████████████████████| 101 kB 1.3 MB/s ta 0:00:01


In [5]:
!python3 -m pip install --index-url https://test.pypi.org/simple/ aries_basic_controller

Looking in indexes: https://test.pypi.org/simple/


In [1]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = "/"
ADMIN_URL = "http://data-agent:8021"

ModuleNotFoundError: No module named 'aries_basic_controller'

In [7]:
# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                           webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, connections=True)

In [8]:
# Spins up server to respond to any webhook events from the agent
import asyncio
asyncio.get_event_loop().run_until_complete(agent_controller.listen_webhooks())
# await agent_controller.listen_webhooks()

In [9]:
# Controllers re-emit webhooks as events through PyPubSub
# Basic state management is handled through defaults but you can choose to pass in custom handlers
custom_handler = []
agent_controller.register_listeners(custom_handler, defaults=True)

In [10]:
# Create Invitation
invite = await agent_controller.connections.create_invitation(alias="Will")

In [11]:
print(invite)

{'connection_id': 'ca5f3332-e5d9-42c4-bad0-db8edc637fb4', 'invitation': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '7b485f18-2d8c-4d54-943a-016b623c33c0', 'recipientKeys': ['8b8rvN2SaxaSiCsgTh7qeU5c14y5QwEkDBCUtNDstStH'], 'label': 'Bob', 'serviceEndpoint': 'http://192.168.65.3:8020'}, 'invitation_url': 'http://192.168.65.3:8020?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiN2I0ODVmMTgtMmQ4Yy00ZDU0LTk0M2EtMDE2YjYyM2MzM2MwIiwgInJlY2lwaWVudEtleXMiOiBbIjhiOHJ2TjJTYXhhU2lDc2dUaDdxZVU1YzE0eTVRd0VrREJDVXRORHN0U3RIIl0sICJsYWJlbCI6ICJCb2IiLCAic2VydmljZUVuZHBvaW50IjogImh0dHA6Ly8xOTIuMTY4LjY1LjM6ODAyMCJ9'}


In [12]:
data_connection_id = invite["connection_id"]
print(data_connection_id)

ca5f3332-e5d9-42c4-bad0-db8edc637fb4


In [ ]:
# Accept Request for Invite created
connection = await agent_controller.connections.accept_request(data_connection_id)
print("ACCEPT REQUEST")
print(connection)

In [ ]:
# Print Connections
connection = await agent_controller.connections.get_connection(data_connection_id)
print("DATA AGENT CONNECTION")
print(connection)

In [ ]:
# Do a trust ping to bring accepted request into "ACTIVE" state
while connection["state"] != "active":
    trust_ping = await agent_controller.connections.trust_ping(data_connection_id, "hello")
    print("TUST PING TO ACTIVATE CONNECTION - DATA -> RESEARCH")
    print(trust_ping)
    time.sleep(5)
    connection = await agent_controller.connections.get_connection(data_connection_id)

In [ ]:
# Print connection list
connection = await agent_controller.connections.get_connection(data_connection_id)
print("DATA AGENT CONNECTION")
print(connection)

In [ ]:
#send some basic messages
message = await agent_controller.connections.send_message(data_connection_id,"hello from data world!")
print("BASIC MESSAGE - DATA -> RESEARCH")
print(message)